In [6]:
from gzip import open # NB: overrides standard open()
import pickle as pkl
import tensorly as tl
from tensorly.base import tensor_to_vec,  partial_tensor_to_vec
import numpy as np
import pandas as pd
import warnings 
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

warnings.simplefilter('ignore') # отключим предупреждения Anaconda
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/88/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/88/y.pkl.gz', 'rb'))

#делим набор на обучающий и тестовый
X_train, X_test, y_train, y_test = train_test_split(
     Xdata_numpy["X"], Ydata, test_size=0.1428 
)
print(X_train)
y_train.shape
# прописываем класс регрессии
from sklearn.base import BaseEstimator
from sklearn.metrics import r2_score
from tensorly.regression.kruskal_regression import KruskalRegressor

class WrappedKruskalRegressor(BaseEstimator):
	def __init__(self, weight_rank = 1, tol = 1e-6, reg_W = 1, n_iter_max = 100):
		self.weight_rank = weight_rank
		self.tol = tol
		self.reg_W = reg_W
		self.n_iter_max = n_iter_max
	
	def fit(self, X, y = None):
		self._model = KruskalRegressor(self.weight_rank, self.tol, self.reg_W, self.n_iter_max)
		self._model.fit(X, y)
		if self._model.n_iterations_ == self.n_iter_max:
			raise Exception('{} did not converge'.format(self))
		return self
	
	def predict(self, X):
		return self._model.predict(X)
	
	def score(self, X, y):
		return r2_score(y, self.predict(X))


#начинаем применять

from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
regressioModel=LinearRegression()
Xmatrix=[]
for i in X_train:
    Xmatrix=pd.DataFrame(X_train[i,:,:], index=str(Xdata_numpy["emission"]),columns=str(Xdata_numpy["excitation"]))

parametrsNames={str(Xdata_numpy["emission"]),str(Xdata_numpy["excitation"])}
gridCought=RandomizedSearchCV(regressioModel, param_distributions=parametrsNames,
                             n_iter=100, cv=5, random_state= 0)

gridCought.fit(X, y_train)

[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.02932944 ... 0.         0.         0.        ]
  [0.         0.         0.02576726 ... 0.         0.         0.        ]
  [0.         0.         0.02804608 ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.02836073 ... 0.         0.         0.        ]
  [0.         0.         0.02262453 ... 0.         0.         0.        ]
  [0.         0.         0.01921441 ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.      

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
ex=tl.to_nympy(estimator.weight_tensor_)